Определение языка (language detection)
--------------------

* **Множество случаев** — тексты на разных языках
* **Множество классов** — языки

### Первый метод: частотные слова

Очень простой и неплохой по качеству метод. Сначала создаем частотный словарь для всех языков и берем самые частотные слова. После этого для каждого текста, который нам надо расклассифицировать, смотрим, частотных слов какого языка в нем больше - тот язык и выбираем.

Метод неплохо работает на текстах длиннее 50 слов и быстро имлементируется. 

В качестве корпусов и текстов для тестирования будем использовать статьи Википедии на разных языках. Скачать Википедию можно различными способами:

* Дампы википедии: https://dumps.wikimedia.org/backup-index.html

* wikiextractor: http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

* annotated_wikiextractor: https://github.com/jodaiber/Annotated-WikiExtractor

* wikipedia: https://pypi.python.org/pypi/wikipedia/

#### Скачаем немного википедии для тестов
Воспользуемся пакетом *wikipedia*:

`pip install wikipedia`

In [3]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [4]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in ('kk', 'uk', 'be', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

kk 100


/usr/lib/python3/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page Будини
Skipping page Аттінья (значення)
Skipping page Луквиця (річка)
Skipping page Сома
Skipping page Оксид вуглецю
Skipping page Арасланово (Мелеузівський район)
Skipping page SSI
uk 93
Skipping page Адамавічы
Skipping page Кавалёнак
Skipping page Стэфан V
Skipping page Лойка
Skipping page Вірус
Skipping page Кучына
be 94
Skipping page Lacerta montana
Skipping page Borovinići
Skipping page Jalisco (homonymie)
Skipping page Borja Fernández
Skipping page Nostradamus (homonymie)
Skipping page GAD
Skipping page Raw and Uncut
fr 93


In [11]:
print(wiki_texts['kk'][0]) # распечатаем пару текстов, чтобы убедиться, что все хорошо
print(wiki_texts['fr'][0])

Мурза (Ница тармағы)
Мурза (Гильдеевка) — Ресейдегі өзен. Свердлов облысы жер аумақтарынан ағып өтеді. Өзен сағасы Ница өзенінің сол жағалауынан 129 км қашықтықта орналасқан. Өзен ұзындығы 61 км-ді құрайды.


 Су реестрінің мәліметтері 
Ресей мемлекеттік су тізілімінің мәліметі бойынша Ертіс су алабы өңіріне жатады, өзеннің сушаруашылық бөлігі — Ница өзені Реж өзені және Нейва өзені өзендерінің қосылған жерінен сағасына дейін. Өзен саласы — Тобыл, өзен алабы — Ертіс.
Ресей су ресурстары федералды агенттігі дайындаған РФ территориясын сушаруашылығы бойынша аудандастыру жөніндегі геоақпараттық жүйе мәліметтері бойынша:
Мемлекеттік су реестріндегі су объектісінің коды — 14010501912111200007224
Гидрологиялық тұрғыдан зерттелу (ГЗ) коды — 111200722
Су алабының коды — 14.01.05.019
ГЗ томының нөмірі — 11
ГЗ бойынша шығарылуы — 2


 Дереккөздер 


 Сыртқы сілтемелер 
Ресей Федерациясы Табиғи ресурстар және экология министрлігі
Huile essentielle de citron
L’huile essentielle de citron (Citrus l

#### Считаем частотный список примерно так:

In [31]:
import codecs
import collections
import sys

def tokenize(text):
    return text.split(' ')

freqs = collections.defaultdict(lambda: 0)

corpus = wiki_texts['kk']
for article in corpus:
    for word in tokenize(article.replace('\n', '').lower()):
        freqs[word] += 1

for word in sorted(freqs, key=lambda w: freqs[w], reverse=True)[:50]:
    print('{}\t{}'.format(freqs[word], word))

256	—
179	және
97	
80	бойынша
71	су
62	дереккөздер
60	мен
59	коды
54	сыртқы
53	сілтемелер
52	мәліметтер
51	бұл
44	жер
41	жалпы
40	болды.
40	өзен
38	жылы
37	саны
36	құрамына
33	аумағы
33	орналасқан.
33	адамды
32	1
32	алып
31	-
30	немесе
30	жатқан
30	тұрғындарының
29	ал
28	ақша
28	халық
27	ж.
27	оның
27	–
27	км²
27	жылғы
27	2
27	деп
26	тапсырма
26	үшін
24	тағы
24	базасындағы
24	ресми
24	болып
24	бір
24	алабы
23	қарасты
22	ол
22	ресей
21	4


#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь можно загружать готовые частотные словари и классифицировать тексты, просто считая количество слов в каждом.

### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [27]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [30]:
freqs = collections.defaultdict(lambda: 0)

corpus = wiki_texts['fr']
for article in corpus:
    for ngram in make_ngrams(article.replace('\n', '').lower()):
        freqs[ngram] += 1

for ngram in sorted(freqs, key=lambda n: freqs[n], reverse=True)[:50]:
    print('{}\t{}'.format(freqs[ngram], ngram))

2782	 de
2352	es 
2228	de 
1520	le 
1482	e d
1271	 le
1238	e l
1202	 la
1171	la 
1056	nt 
1013	re 
985	ent
921	ion
908	on 
904	et 
825	en 
801	 co
794	s d
784	 et
748	 en
746	que
710	tio
673	e p
668	ne 
666	les
646	 po
643	est
637	 pa
636	ns 
624	lle
621	ur 
618	e s
592	er 
587	des
587	 pr
585	 l'
579	t d
569	e c
561	ant
559	ue 
550	par
548	il 
545	 à 
541	e e
538	 du
535	st 
527	s, 
525	ati
495	 es
494	te 


#### Нужно сделать это для каждого языка и отфильтровать повторяющиеся

Теперь, как и в предыдущем методе, можно загружать готовые частотные словари n-грамм и классифицировать тексты, просто подсчитывая частотные n-граммы в каждом.